___
##### $Name:\,\color{blue}{Christopher\,J.\,Watson\,,Nathan\,Edwards\,,Paul\,Thai}$
##### $School:\,\color{blue}{Marcos\,School\,of\,Engineering,\,University\,of\,San\,Diego}$
##### $Class:\,\color{blue}{AAI\,511-\,Neural\,Networks\,and\,Learning}$
##### $Assignment:\,\color{blue}{MSAAI\,Final\,Project}$
##### $Date:\,\color{blue}{8/15/2023}$
___

### Libraries
____

In [1]:
# Libraries 

# File and Data Handling
import os

# Data Visualization and Numerical Computing
import matplotlib.pyplot as plt
import numpy as np

# MIDI Processing
import mido

# Machine Learning and Deep Learning
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn, optim

# Machine learning Metrics and Evaluations
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix

### Self-Defined Classes
___

In [2]:
# Midi Transcriber
class MidiTranscriber():
    def __init__(self):
        pass

    def Sequence(self, track):
        result = []
        notes = {}
        time = 0
        final_result = []
        data = [] 
        state = [0] * 128
        tick_counter = 0
        
        tick_interval = 20
        
        for i in range(16):
            data.append(state)
            
        for msg in track:
            time += msg.time
            tick_counter += msg.time
            
            if(tick_counter > tick_interval):
                final_result.append(data)
                tick_counter = 0
            
            if msg.type == 'note_on':
                if msg.velocity != 0:
                    past = notes.get(msg.note)
                    if(past == None):
                        past = []
                    notes[msg.note] = past + [time , 1] 
                    result.append([time, msg.note, msg.velocity,  msg.channel, msg.type])
                    data[msg.channel][msg.note] = msg.velocity
                else:
                    pass
            if msg.type == 'note_off':
                past = notes.get(msg.note)
                if(past == None):
                    past = []
                notes[msg.note] = past + [time , 0]
                result.append([time ,msg.note, msg.velocity ,  msg.channel, msg.type])
                data[msg.channel][msg.note] = 0
                
        return result, data, final_result
        
        
    def BuildArray(self,midi):
        Lenghth = [len(tr) for tr in midi.tracks]
        track_length = max(Lenghth)
        min_track =  track_length/10 #Only accept tracks 1/10 of length.
        all_arys = []
        training = []
        for track in range(len(midi.tracks)):
            out = []
            valid_track = (len(midi.tracks[track]) > min_track)
            if(valid_track):
                track = midi.tracks[track]
                res, data, final_result = self.Sequence(track)
                all_arys.append(res)
                training.append(data)

        return all_arys, data, final_result


In [3]:
scriber = MidiTranscriber()

In [4]:
def Parser(composer, path):
    midis = os.listdir(path)
    
    pair = []
    
    for file in midis:
        if(file == ".DS_Store"):
            continue 
        midis_path = path + "/" + file
        #print(midis_path)
        
        midi = mido.MidiFile(midis_path)
        #print(len(midi.tracks[0]) + len(midi.tracks[1]) + len(midi.tracks[2]))
        #print(len(midi.tracks[0]))
        arr, data, final_result = scriber.BuildArray(midi)
        
        
        sequence_size = 100
        
        div = len(final_result) / sequence_size
        last = 0
        for i in range(int(div)):

            clip = final_result[last:sequence_size*int((i+1))]
            last = int(sequence_size)*(i+1)
            
            x = clip
            y= composer
 
            pair.append([x,y])
    
            if(not len(x) == 100):
                print(len(x))
        
    return arr, data, final_result, pair
        


In [9]:
class Dataset(Dataset):
     
    def enum(self,y):
        composers = ["bach", "bartok", "byrd", "chopin", "handel", "hummel", "mendelssohn","mozart", "schumann"]
        #print(y)
        for i in range(len(composers)):
            if composers[i] == y:
                slot = i
        out = [0] * 9
        out[slot] = 1
        
        return out
        
    def __init__(self, pair):
        self.pair = pair
    def __len__(self):
        return len(self.pair)
    
    def __getitem__(self,idx):
        
        x = torch.tensor(self.pair[idx][0])
        y = torch.tensor(self.enum(self.pair[idx][1]))
        return x,y
        

### Helper Functions
___

In [5]:
# Creating the pairs
def create_pairs(dataset_path, composers):
    # Create a list to hold the data
    pairs = []

    # Iterate through the dataset, grad the data, and append to list
    for composer in composers:
        path = dataset_path +"/"+ composer
        print(composer)
        arr, data, final_result, pair = Parser(composer, path)
        pairs.append(pair)

    return pairs

# Combine all the data into one single
def combine_all(pairs):
    # Create new list to hold the new data
    new_list = []

    # Iterate through the pairs and combine them
    for i in pairs:
        for j in i:
            new_list.append(j)
            
    return new_list

### Preprocessing
___

In [6]:
# Set our train and test dataset
dataset_train = "Composer_Dataset/NN_midi_files_extended/train"
dataset_test = "Composer_Dataset/NN_midi_files_extended/test"

# List of our composers
composers = ['bach', 'bartok', 'byrd', 'chopin', 'handel', 'hummel', 'mendelssohn', 'mozart', 'schumann']

# Create the train and test pairs
train_pairs = create_pairs(dataset_path=dataset_train,composers=composers)
test_pairs = create_pairs(dataset_path=dataset_test, composers=composers)


bach
bartok
byrd
chopin
handel
hummel
mendelssohn
mozart
schumann
bach
bartok
byrd
chopin
handel
hummel
mendelssohn
mozart
schumann


In [7]:
# Train and Test pair
train_pair = combine_all(train_pairs)
test_pair = combine_all(test_pairs)

In [10]:
# Creating Train and Test Dataset 
train_data = Dataset(train_pair)
test_data = Dataset(test_pair)

In [11]:
# Validate the shape of an arbitrary location to ensure it is the shape we desire
print(train_data[1000][0].shape)
print(test_data[100][0].shape)

torch.Size([100, 16, 128])
torch.Size([100, 16, 128])


In [12]:
# Load in our train and test data with dataloader 
train_dataloader = DataLoader(train_data,batch_size=3,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=3, shuffle=False)

### Model Creation
___

In [13]:
# Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(100, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 2 * 64, 128) 
        self.fc2 = nn.Linear(128, 9)  

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# Creating the Long Short Term Memory model class
class LSTM(nn.Module):
    # create the constructor of our RNN
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM,self).__init__()
        # self initialize the hidden size LSTM, and fully connected layer
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self,x):
        # Initialize the hidden state and cell state
        hidden_state = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        cell_state = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        # grabbing the ouput of the hidden state
        output_state, _ = self.lstm(x, (hidden_state, cell_state))

        # pass output state into through the fully connected layer
        output_state = self.fc(output_state[:, -1, :])

        return output_state


In [14]:
# Create model object
model = CNN() 

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

### Training
___

In [20]:
# Training loop
num_epochs = 6
for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        inputs = inputs.to(torch.float)
        labels = labels.to(torch.float)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the average loss for the epoch
    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}")

print("Training complete!")

Epoch [1/6] Loss: 5.4209
Epoch [2/6] Loss: 5.4199
Epoch [3/6] Loss: 5.4193
Epoch [4/6] Loss: 5.4189
Epoch [5/6] Loss: 5.4187
Epoch [6/6] Loss: 5.4185
Training complete!


### Evaulation and Results
___

In [21]:
# Evaluating the model -  rough outline not done yet. 
model.eval()

prediction_list = list()
labels_list = list()

# Disable gradient computation to save memory
with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs = inputs.to(torch.float)
        labels = labels.to(torch.float)
        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        prediction_list.extend(predicted.numpy())
        labels_list.extend(labels.numpy())

print("Prediction List", prediction_list)
print("Labels", labels)

# Finding accuracy, precision, f1 score and confusion matrix
# accuracy = accuracy_score(labels_list, prediction_list)
# precision = precision_score(labels_list, prediction_list, average='weighted')
# f1_score = f1_score(labels_list, prediction_list, average='weighted')
# confusion_mtrx = confusion_matrix(labels_list, prediction_list)

Prediction List [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 4, 4, 4, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,